In [2]:
#import necessary package and collect initial variables
import openmdao.api as om
import pandas as pd
import numpy as np

In [ ]:
#define discipline group, get constants and call analyses 
class sys(om.Group):
    def initialize(self):
        
        #read all excel constants        
        initial_vars = pd.read_excel('constants.xlsx')
        
        #sort for just the systems constants
        all_sys_vars = np.where(initial_vars['sys'] == True, [initial_vars['variable name'], initial_vars['value'], initial_vars['description']], None)
        
        #create a DataFrame for the systems variables to pull from in individual analyses
        all_sys_vars = pd.DataFrame(all_sys_vars).dropna(axis = 1).transpose()
        
        #pass variables into the correct openMDAO format, passing name, description and value
        i=0
        for variable in all_sys_vars[0]:
            value = all_sys_vars.iloc[i][1]
            description = all_sys_vars.iloc[i][2]
            self.options.declare(variable, default = value, desc = description)
            i+=1
        i=0
        
    def setup(self):
        #set analysis (subsystem) names and what inputs they take and give
        self.add_subsystem('fuel_mass_calc', fuel_mass_calc())
        self.add_subsystem('tanks', tanks())
        self.add_subsystem('pipes', pipes())
        self.add_subsystem('engine', engine())
        self.add_subsystem('control_surface', control_surface())
        self.add_subsystem('systems_roundup', systems_roundup())
        #can optionally promote the variables at this step
        
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually)
        self.promotes('fuel_mass_calc',any=['*'])
        self.promotes('tanks',any=['*'])
        self.promotes('pipes',any=['*'])
        self.promotes('engine',any=['*'])
        self.promotes('control_surface',any=['*'])
        self.promotes('systems_roundup',any=['*'])

In [16]:
class fuel_mass_calc(om.ExplicitComponent):
    def initialize(self):
        #take constants from group a level above
        self.options.declare('eta',prob.model.sys.options['eta'])
        self.options.declare('R',prob.model.sys.options['range'])
        self.options.declare('SFC',prob.model.sys.options['TSFC'])
        self.options.declare('M_initial',prob.model.sys.options['M_initial'])
        
        
        
    def setup(self):
        #define inputs (variables to come from other disciplines) and default values
        self.add_input('CD', val = 0.02)
        self.add_input('CL', val = 0.2)
        
        self.add_output('fuel_mass')
        
    def compute(self,inputs,outputs):
        #define variables used in equations and where they come from (constant or input)
        eta = self.options['eta']
        R = self.options['R']*1000
        SFC = self.options['SFC']
        W0 = self.options['M_initial']
        
        CD = inputs['CD']
        CL = inputs['CL']
        
        #fuel mass fraction derived from breguet range equation
        Wf = np.exp(-R*SFC/(eta*CL/CD))
        
        outputs['fuel_mass'] = W0*Wf

In [ ]:
class tanks(om.ExplicitComponent):
    def initialize(self):
        #take constants from group a level above
        self.options.declare('LH2_rho',prob.model.sys.options['LH2_rho'])
        self.options.declare('tank_rho',prob.model.sys.options['tank_rho'])
        self.options.declare('insul_rho',prob.model.sys.options['insul_rho'])
        self.options.declare('tank_t',prob.model.sys.options['tank_t'])
        self.options.declare('insul_t',prob.model.sys.options['insul_t'])
        self.options.declare('L_cock',prob.model.sys.options['L_cock'])
        self.options.declare('L_ce',prob.model.sys.options['L_ce'])
        self.options.declare('L_c1',prob.model.sys.options['L_c1'])
        self.options.declare('pp_mass',prob.model.sys.options['pp_mass'])
        
    def setup(self):
        #define inputs (variables to come from other disciplines) and default values
        self.add_input('fuel_mass', val = 30000)
        self.add_input('fuselage_diameter', val = 8)
        
        self.add_output('tank1_mass_full')
        self.add_output('tank1_mass_empty')
        self.add_output('tank2_mass_full')
        self.add_output('tank2_mass_empty')
        self.add_output('CGtp')
        self.add_output('tank1_x')
        self.add_output('tank2_x')
        
    def compute(self,inputs,outputs):
        LH2_rho = self.options['LH2_rho']
        tank_rho = self.options['tank_rho']
        insul_rho = self.options['insul_rho']
        t = self.options['tank_t']
        insul_t = self.options['insul_t']
        L_cock = self.options['L_cock']
        L_ce = self.options['L_ce']
        L_c1 = self.options['L_c1']
        pp_mass = self.options['pp_mass']
        
        
        f_mass = inputs['fuel_mass']
        R = inputs['fuselage_diameter']/2 #outer radius of tank
        
        
        f_vol = f_mass*(1.072/LH2_rho) #calculate fuel volume, uses allowance factor from book
        
        tV1 = f_vol*3/5 #ratios of tank volume using book values of roughly 3:2 forward to aft
        tV2 = f_vol*2/5
        
        R_i = R-insul_t #inner radius of tanks
        
        tL1_c = (tV1-((4/3)*(np.pi*R_i**3)/1.6))/(np.pi*R_i**2) #length of cylindrical section of tank 1
        tL2_c = (tV2-((4/3)*(np.pi*R_i**3)/1.6))/(np.pi*R_i**2) #ellipse a/b ratio of 1.6 is used from book for end sections
        
        tL1 = tL1_c+2*(R_i/1.6) #total lengths of tanks 
        tL2 = tL2_c+2*(R_i/1.6)
        
        A1 = (np.pi*2*R_i*tL1_c)+4*np.pi*(((R_i**2)**1.6+2*((R_i**2/1.6)**1.6))/3)**(1/1.6) #surface area of tanks
        A2 = (np.pi*2*R_i*tL2_c)+4*np.pi*(((R_i**2)**1.6+2*((R_i**2/1.6)**1.6))/3)**(1/1.6) #uses surface area of ellipsoid for end parts
        
        tm1 = A1*(t*tank_rho+insul_t*insul_rho) #empty tank masses
        tm2 = A2*(t*tank_rho+insul_t*insul_rho)
        
        outputs['tank1_mass_empty'] = tm1
        outputs['tank2_mass_empty'] = tm2
        fulltank1 = tm1+f_mass*0.6
        fulltank2 = tm2+f_mass*0.4
        outputs['tank1_mass_full'] = fulltank1
        outputs['tank2_mass_full'] = fulltank2
        
        t_c_m = fulltank1 + fulltank2 + pp_mass #total full mass of tanks and payload
        
        #centre of gravity of tanks and payload using configuration tank1->economy_cabin->tank2->1st class to bring CG forward
        CGtp = ((L_cock+0.5*tL1)*fulltank1+(0.8*pp_mass*(L_cock+tL1+(L_ce/2)))+fulltank2*(L_cock+tL1+L_ce+0.5*tL2)+(0.2*pp_mass*(L_cock+tL1+L_ce+tL2+0.5*L_c1)))/t_c_m 
        outputs['tank1_x'] = L_cock+0.5*tL1
        outputs['tank2_x'] = L_cock+tL1+L_ce+0.5*tL2
        outputs['CGtp'] = CGtp

        
        #could optionally add tank insulation width study

In [ ]:
class pipes(om.ExplicitComponent):
    def initialize(self):
        self.options.declare('insul_kappa',prob.model.sys.options['insul_kappa'])
        self.options.declare('insul_rho',prob.model.sys.options['insul_rho'])

        
    def setup(self):
        self.add_input('engine_x', val = 30)
        self.add_input('engine_y', val = 10)
        self.add_input('tank1_x', val = 8)
        self.add_input('tank2_x', val = 40)        
        
    def compute(self, inputs, outputs):
        #find maximum length of pipe (from tank to engine) and determine the heat gain with different insulation thicknesses
        #determine total pipe length and mass to determine pipe CG
        
        maxpipe1_2 = abs(inputs['tank1_x'] - inputs['engine_x'])+abs(inputs['engine_y']) #longest route will be through cross-feed, tank to alternate engine
        maxpipe2_1 = abs(inputs['tank2_x'] - inputs['engine_x'])+abs(inputs['engine_y'])
        max_p = max(maxpipe1_2,maxpipe2_1) #maximum pipe length
        
        #find thermal relationship and relate to pipe length to determine pipe insulation width & mass and then CG
        
        
        

In [ ]:
class engine(om.ExplicitComponent):
    def initialize(self):
        self.options.declare('T',prob.model.sys.options['T'])
        self.options.declare('T_m',prob.model.sys.options['T_m'])
    
    def setup(self):
        self.add_input('fuselage_diameter', val = 8)
        self.add_input('wingspan', val = 80)
        self.add_output('engine_mass', val = 1500)
        
        
    def compute(self,inputs,outputs):
        #engine size based off the original constraint diagram calculation
        T = self.options['T']
        T_m = self.options['T_m']
        engine_mass = T_m*T/2
        outputs['engine_mass'] = engine_mass
        
        #decide on engine position, will be an implicit function with the pipes calculation in previous cell as well as the bending moment from structures
        #ensure to add bounds so the engines are not too far or too close

In [ ]:
class control_surface(om.ExplicitComponent):
    pass
    #box for control surface requirement calculations 
    #calculate mass of systems based on load requirements 
    #use positions and mass to calculate a CG
    

In [ ]:
class systems_roundup(om.ExplicitComponent):
    pass
    #add additional components that are a fixed mass (and probably position too)
    #collate other systems analyses to obtain one single systems mass and CG to pass to structures